In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from glob import glob
from pprint import pprint

sys.path.append("/home/phornauer/Git/axon_tracking/")
from axon_tracking import template_extraction as te

In [ ]:
te_params = dict()
te_params['n_jobs'] = 16 #Number of cores to use for waveform extraction
te_params['filter_band'] = 150 #Either float for the highpass filter frequency or list for the bandpass filter frequencies
te_params['overwrite'] = False #Flag if templates should be recalculated if already existing
te_params['max_spikes_per_unit'] = 1000 #Maximum number of spikes to be used for template extraction

qc_params = dict()
qc_params['min_n_spikes'] = 1500 #Minimum number of spikes to be detected for a unit for template extraction to take place
qc_params['exclude_mua'] = True #Exclude units that were labelled multi unit activity by kilosort
qc_params['use_bc'] = False #Use bombcell for QC (has to be run separately)

We only need a list of folders containing the sorting results (sorting_list) and can then infer the recording paths automatically. Change root_path and path_pattern so that they represent the sortings you want to analyze. 
Use wildcards (*) where appropriate, but still try to be as specific as possible to prevent ambiguities.

In [ ]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/" # Fixed path root that all recordings have in common
path_pattern = ["AxonScan", "23*",  "T002443","AxonTracking","w*","sorter_output"] # Variable part of the path, where we collect all possible combinations using wildcards (*). It is still recommended to be as specific as possible to avoid ambiguities.

full_path = os.path.join(root_path, *path_pattern)
sorting_list = glob(full_path)
sorting_list.sort()
print(f'Found {len(sorting_list)} sorting paths matching the description:\n{full_path}\n')
#pprint(sorting_list)

Optional: Check the sorting list and only include a selection of the sortings

In [ ]:
pprint(sorting_list)

In [ ]:
sorting_list = sorting_list[:1] # For testing purposes, we only use the first recording

Now we can run the actual template extraction:

In [ ]:
te.extract_templates_from_sorting_list(sorting_list, qc_params, te_params)

We can also have a quick look at the template we just extracted:

In [ ]:
from axon_tracking import visualization as vis #Separate as it is not needed for the template extraction

In [ ]:
root_path = "/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Torsten_2/241010/T002523/AxonTracking/well000/sorter_output/"
plot_path = os.path.join(root_path, 'templates')

vis.plot_template_overview(plot_path)